# CCXT 是什么

* ccxt(crypto currency exchange trading)

* 最好的加密货币交易库， 包含了近百个交易所api。

* 支持 Python、JavaScript、PHP 三种语言调用。

我用python来调用ccxt, 数据分析，策略设计和信号生成。

安装非常简单：

pip install ccxt

ccxt官网：
https://ccxt.readthedocs.io/en/latest/manual.html

### 用ccxt来熟悉交易所

* 前提假设是你对python有一定的熟悉，知道数据处理常用库的基本使用（pandas, numpy等），同时也知道如何根据自己的习惯安装新的库（例如这里用到的ccxt)



In [40]:
import ccxt
ccxt.exchanges #展示所有交易所名称

In [ ]:
exchange = ccxt.binance() #设置币安为提取数据的交易所，也可以选其他
exchange.load_markets()

In [41]:
## 币安目前交易所有币种
all_coins = list(exchange.load_markets().keys())
all_coins

In [32]:
# 抓取并整理btc 日线数据，1000天
BTC_df = pd.DataFrame(exchange.fetch_ohlcv('BTC/USDT', timeframe='1d', limit=1000)) 
BTC_df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol']
BTC_df['Datetime'] = BTC_df['Datetime'].apply(
   lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))

BTC_df

,Datetime,Open,High,Low,Close,Vol
0,2020-06-12 00:00:00,9278.88,9557.12,9232.51,9465.13,50119.066932
1,2020-06-13 00:00:00,9464.96,9494.73,9351.00,9473.34,27759.784851
2,2020-06-14 00:00:00,9473.34,9480.99,9245.00,9342.10,30055.506608
3,2020-06-15 00:00:00,9342.10,9495.00,8910.45,9426.02,86107.924707
4,2020-06-16 00:00:00,9426.05,9589.00,9373.09,9525.59,52052.446927
...,...,...,...,...,...,...
995,2023-03-04 00:00:00,22354.34,22410.00,22157.08,22346.57,121257.381320
996,2023-03-05 00:00:00,22346.57,22662.09,22189.22,22430.24,154841.757860
997,2023-03-06 00:00:00,22430.24,22602.19,22258.00,22410.00,203751.829570
998,2023-03-07 00:00:00,22409.41,22557.91,21927.00,22197.96,292519.809120


## 通过交易量，观察一周里面哪几天最活跃，那几天最不活跃？

In [33]:
# Convert the datetime column to datetime format
BTC_df['Datetime'] = pd.to_datetime(BTC_df['Datetime'])
# Group by the datetime column and sum up the vol column
grouped_df = BTC_df.groupby(BTC_df['Datetime'].dt.day_name()).agg({'Vol': 'sum'})
grouped_df.sort_values(by='Vol', ascending=False)

,Vol
Datetime,
Wednesday,1.785334e+07
Tuesday,1.750807e+07
Friday,1.736498e+07
Thursday,1.720165e+07
Monday,1.700853e+07
Sunday,1.162154e+07
Saturday,1.147771e+07


* 看起来周末最不活跃，是不是说明比特币机构玩家增多，而机构在周末不交易，导致周末交易量相比平时少了很多？

# 比特币在一天的哪个时间交易最活跃，哪个时间最不活跃？

* utc 时间为准，北京时间则加8，美国时间减4

In [39]:
BTC_df = pd.DataFrame(exchange.fetch_ohlcv('BTC/USDT', timeframe='1h', limit=1000)) 
BTC_df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol']
BTC_df['Datetime'] = BTC_df['Datetime'].apply(
   lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))
# Convert the datetime column to datetime format
BTC_df['Datetime'] = pd.to_datetime(BTC_df['Datetime'])
# Group by the datetime column and sum up the vol column
grouped_df = BTC_df.groupby(BTC_df['Datetime'].dt.hour).agg({'Vol': 'sum'})
grouped_df.sort_values(by='Vol', ascending=False)

,Vol
Datetime,
15,896007.63867
14,796624.08651
16,766331.79858
19,637325.43608
13,633558.58631
20,630100.79044
17,571280.46687
18,557967.47876
12,504635.32702


## 批量下载k线数据
* 下面的代码可以下载并保存币安过去所有期货合约币种的日线价格和交易量的数据，每个币1000条，两年多的数据（如果有两年的话）
* 你可以修改周期-tineframe（1d, 1h, 4h, 15min）,还有 defaultType (future-期货合约，spot现货) 来下载自己想要的数据
* 日线数据大概需要20分钟左右，具体速度与网速有关
* 频繁拉取币安数据会被封IP, 记得使用VPN

In [1]:
import numpy as np
import pandas as pd

import time
import dateutil
from datetime import datetime

exchange = ccxt.binance({
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',  # or 'spot'
    }})

all_coins = list(exchange.load_markets().keys())

all_candles = []

for symbol in all_coins:
    
    df = pd.DataFrame(exchange.fetch_ohlcv(symbol, timeframe='1d', limit=1000))
    
    df['symbol'] = symbol
    
    all_candles.append(df)
    
all_candles = pd.concat(all_candles)

all_candles.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Symbol']

all_candles['Datetime'] = all_candles['Datetime'].apply(
   lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))

all_candles.to_csv("binance_coins_daily_candlesticks.csv") #20分钟完成